In [49]:
from io import StringIO

import matplotlib.pyplot as plt
from scipy.io import arff
import seaborn as sns
from loguru import logger
import yaml

from datetime import datetime
import polars as pl
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

# MODEL
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import (
    LabelEncoder, 
    StandardScaler,
    label_binarize
)
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_recall_fscore_support,
    balanced_accuracy_score,
    roc_auc_score,
    roc_curve
)

# Save variables for model
import joblib
import json

import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# PERSONAL FUNCTIONS
from utils import *
from models.main import *
from models.optimizer import ViterbiLiteDecoder
from functions.windows import create_feature_windows # creación de ventanas e ingenieria de características
from functions.build_window_raw import create_raw_windows_250_timesteps_robust

In [50]:
# dc = {'sit': 96.42857142857143, 'stand': 0.0, 'walk': 98.21958456973294, 'type': 96.15384615384616, 'eat': 100.0}

In [51]:
dict_preds = {
    'Eat': {'y': [], 'y_pred': []},
    'Stand': {'y': [], 'y_pred': []},
    'Walk': {'y': [], 'y_pred': []},
    'Sit': {'y': [], 'y_pred': []},
    'Type': {'y': [], 'y_pred': []}
}

In [52]:
dic_result = {
            'Eat': {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []}, 
            'Walk': {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
            'Sit': {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
            'Stand': {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
            'Type': {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []}
        }

In [53]:
loaded = tf.saved_model.load(r"F:\UPC\Tesis\HARbit-Model\src\cnn_temporal_20_epochs_93\saved_model")
infer = loaded.signatures["serving_default"]
label_encoder = joblib.load(r'F:\UPC\Tesis\HARbit-Model\src\cnn_temporal_20_epochs_93\label_encoder.joblib')


In [54]:
path_base = r"F:\UPC\Tesis\HARbit-Model\src\data\real-data"
archive_date = [archive for archive in os.listdir(r"F:\UPC\Tesis\HARbit-Model\src\data\real-data") if '.json' not in archive]


for data in archive_date:
    path_data = os.path.join(path_base, data)
    file_data = os.listdir(path_data)

    target = data.split('-')[0].title()
    for file in file_data:

        file_path = os.path.join(path_data, file)
        
        with open(file_path, 'rb') as file:
            data = json.load(file)

        gyro_df = data['gyro']
        accel_df = data['accel']

        accel_temp = pl.DataFrame(accel_df)
        gyro_temp = pl.DataFrame(gyro_df)

        
        accel_temp = accel_temp.with_columns(pl.lit('A').alias('Usuario'))
        gyro_temp  = gyro_temp.with_columns(pl.lit('A').alias('Usuario'))

        accel_temp = accel_temp.with_columns(pl.lit(target).alias('gt'))
        gyro_temp = gyro_temp.with_columns(pl.lit(target).alias('gt'))

        df_accel = normalize_columns(accel_temp,
                            user_col_name  = "Usuario", 
                            timestamp_col_name = "timestamp", 
                            label_col_name = "gt", 
                            x_col_name = "x", 
                            y_col_name = "y", 
                            z_col_name = "z"
                        )

        df_gyro = normalize_columns(gyro_temp, 
                            user_col_name  = "Usuario", 
                            timestamp_col_name = "timestamp", 
                            label_col_name = "gt", 
                            x_col_name = "x", 
                            y_col_name = "y", 
                            z_col_name = "z"
                        )
        
        df_accel = convert_timestamp(df_accel)
        df_gyro = convert_timestamp(df_gyro)

        X_all, y_all, subjects_all, metadata_all = create_raw_windows_250_timesteps_robust(
            df=df_accel,
            window_seconds=5,
            overlap_percent=50,
            sampling_rate=20,
            target_timesteps=100,
            min_data_threshold=0.8,  # 80% mínimo de datos
            max_gap_seconds=1.0      # Máximo 1 segundo de gap
        )

        X_tensor = tf.constant(X_all, dtype=tf.float32)
        y_pred = infer(X_tensor)
        y_all = label_encoder.transform(y_all)

        # Ejemplo: y_pred con probabilidades
        y_pred_probs = list(y_pred.values())[0].numpy()

        # Convertir a clases predichas (índice del máximo)
        y_pred_classes = np.argmax(y_pred_probs, axis=1)


        # Comparar con y_real (asegúrate de que y_real tenga etiquetas 0..N-1)
        acc = accuracy_score(y_all, y_pred_classes)
        
        precision, recall, f1, _ = precision_recall_fscore_support(
                y_all, y_pred_classes, average='macro', zero_division=0
            )
        
        dic_result[target]['Accuracy'].append(acc*100)
        dic_result[target]['F1-score'].append(f1*100)
        dic_result[target]['Precision'].append(precision*100)
        dic_result[target]['Recall'].append(recall*100)

        dict_preds[target]['y'].extend(y_all)
        dict_preds[target]['y_pred'].extend(y_pred_classes)        

🔧 Configuración de ventanas RAW ROBUSTA:
  Duración: 5s
  Timesteps objetivo: 100
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Umbral mínimo de datos: 80.0%
  Máximo gap permitido: 1.0s
  Duración de ventana: 5s
  Paso entre ventanas: 2.50s
👤 Usuario A, Actividad Eat: 6531 muestras
   Duración total: 150.3s
  ✅ Creadas 59 ventanas válidas

📊 RESUMEN DE VALIDACIÓN:
  Ventanas intentadas: 59
  Ventanas creadas: 59
  Tasa de éxito: 100.0%

📊 RESULTADO FINAL (ROBUSTO):
  Forma de X: (59, 100, 3)
  Forma de y: (59,)
  Total ventanas: 59
  Usuarios únicos: 1
  Actividades únicas: ['Eat']
🔧 Configuración de ventanas RAW ROBUSTA:
  Duración: 5s
  Timesteps objetivo: 100
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Umbral mínimo de datos: 80.0%
  Máximo gap permitido: 1.0s
  Duración de ventana: 5s
  Paso entre ventanas: 2.50s
👤 Usuario A, Actividad Eat: 6488 muestras
   Duración total: 150.4s
  ✅ Creadas 59 ventanas válidas

📊 RESUMEN DE VALIDACIÓN:
  Ventanas intentadas: 59
  

In [55]:
for activity in dict_preds.keys():

    y_true = dict_preds[activity]['y']
    y_pred = dict_preds[activity]['y_pred']
    
    acc = accuracy_score(y_true, y_pred)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average='macro', zero_division=0
        )
    print(f"{activity}")
    print(f"Accuracy: {acc*100}")
    print(f"F1-score: {f1*100}")
    print(f"Precision: {precision*100}")
    print(f"Recall: {recall*100}")
    print("-" * 50)

Eat
Accuracy: 82.44274809160305
F1-score: 30.125523012552303
Precision: 33.33333333333333
Recall: 27.480916030534353
--------------------------------------------------
Stand
Accuracy: 0.0
F1-score: 0.0
Precision: 0.0
Recall: 0.0
--------------------------------------------------
Walk
Accuracy: 94.35146443514645
F1-score: 19.418729817007534
Precision: 20.0
Recall: 18.87029288702929
--------------------------------------------------
Sit
Accuracy: 89.44954128440367
F1-score: 47.21549636803874
Precision: 50.0
Recall: 44.72477064220183
--------------------------------------------------
Type
Accuracy: 97.65886287625418
F1-score: 49.407783417935704
Precision: 50.0
Recall: 48.82943143812709
--------------------------------------------------


In [56]:
from sklearn.metrics import classification_report

y_true_global = []
y_pred_global = []

for activity in dict_preds.keys():
    y_true_global.extend(dict_preds[activity]['y'])
    y_pred_global.extend(dict_preds[activity]['y_pred'])

labels = np.unique(y_true_global)  # clases presentes en los datos
target_names = label_encoder.inverse_transform(labels)

print(classification_report(
    y_true_global, y_pred_global,
    labels=labels,
    target_names=target_names
))

              precision    recall  f1-score   support

         Eat       0.31      0.82      0.45       131
         Sit       0.88      0.89      0.89       218
       Stand       0.00      0.00      0.00       205
        Type       1.00      0.98      0.99       299
        Walk       1.00      0.94      0.97       956

   micro avg       0.85      0.83      0.84      1809
   macro avg       0.64      0.73      0.66      1809
weighted avg       0.82      0.83      0.82      1809



In [57]:
accuracy_score(y_true_global, y_pred_global) * 100

82.75290215588723